In [57]:
import os
import pickle
import pickle
from heapq import heapify,heappop,heappush
import random

### bien doi txt ban dau thanh cac shingle roi save pickle

In [58]:
#! extract data and data understanding
FolderPath = os.path.abspath("test")

#! check files in folder:
allFiles = os.listdir(FolderPath)
filePathList = [FolderPath + "/" + i for i in allFiles]
# filePathList


In [60]:

#! check xem trong moi file co bao nhieu tu? 
cnt = 0
for file in filePathList:
    f = open(file,"r")
    words = f.read().split(" ")
    #print(len(words))

    temp = set()
    
    for index in range(0, len(words) - 2):
        #! shingle duoc chia thanh cac tu voi len = 3
        #! vd: This is my shingle: the first successful
        shingle = words[index] + " " + words[index+1] + " " \
        + words[index+2]
        if shingle not in shingleDict.keys():
            #! cac shingle phai unique 
            shingleDict[shingle] = cnt
            cnt += 1
        temp.add(shingleDict[shingle])
        
    docShingleDict[file.split("/")[-1].split(".")[0]] = temp
    



In [61]:
# docShingleDict["file40"]

### SAVE PICKLE

In [62]:
# print(len(shingleDict))
# output = open("docShingleDict.pkl", 'wb')
# pickle.dump(docShingleDict, output)
# output.close()

### MINHASH

#### unpickle de lay shingle

In [63]:
totalShingles = 23880 
pklHandler = open("docShingleDict.pkl", 'rb')
docShingleDict= pickle.load(pklHandler)
pklHandler.close()

In [64]:
#! muc tieu luc nay la convert characteristic matrix ve signature matrix
def findRandomNos(k):
  '''
  return a list of k random unique int
  '''
  randList = []
  randIndex = random.randint(0, totalShingles -1) 
  randList.append(randIndex) 

  while k>0:
    while randIndex in randList: #! dam bao rang 
      randIndex = random.randint(0, totalShingles-1) 
    
    randList.append(randIndex)
    k = k-1
  return randList

#! NUMBER OF ROW IN SIGNATURE MATRIX IS 25
randomNoA = findRandomNos(25)
randomNoB = findRandomNos(25)

#! THIS IS SIGNATURE MATRIX
docLowestShingleID = {} 
for doc in docShingleDict.keys():
  
  shingleIDSet = docShingleDict[doc]

  
  lowestShingleID = []
  for x in range(0,25):
    listFx = [] 
    for shingleID in shingleIDSet:
      hashValue = (randomNoA[x] * shingleID + randomNoB[x]) % totalShingles #! THIS IS MY HASH FUNCTION!
      listFx.append(hashValue)
    heapify(listFx) 
    lowestShingleID.append(heappop(listFx)) 
  docLowestShingleID[doc] = lowestShingleID 
  #! luc nay docLowestShingleID se co value la hash function min nhat. 
  '''
                        | file00 | file01 | file02 | ... 
number1(unique shingle) | (hashValue)
--------------------------------------------------------
number2(unique single)  |
--------------------------------------------------------
  '''

def getFileNo(x):
  if x<10:
    x = "0" + str(x)
  else:
    x = str(x)
  
  return x

In [65]:
estimateMatrix = []
for x in range(0,100):
  doc1 = "file"+getFileNo(x)
  doc1LowestShingles = docLowestShingleID[doc1] #! we can call doc1LowestShingles is minhash signature
  col = []
  for y in range(0,100):
    doc2 = "file"+getFileNo(y)
    doc2LowestShingles = docLowestShingleID[doc2]
    count = 0
    for i in range(0,25):
      #! track number of value minHash signature equal
      if doc1LowestShingles[i] == doc2LowestShingles[i]:
        count = count + 1
        
    col.append(count/25)
  estimateMatrix.append(col)

In [66]:
estimateMatrix = []
for x in range(0,100):
  doc1 = "file"+getFileNo(x)
  doc1LowestShingles = docLowestShingleID[doc1] #! we can call doc1LowestShingles is minhash signature
  col = []
  for y in range(0,100):
    doc2 = "file"+getFileNo(y)
    doc2LowestShingles = docLowestShingleID[doc2]
    count = 0
    for i in range(0,25):
      #! track number of value minHash signature equal
      if doc1LowestShingles[i] == doc2LowestShingles[i]:
        count = count + 1
        
    col.append(count/25)
  estimateMatrix.append(col)

#print("This is my estimateMatrix: {}".format(estimateMatrix))
  
print ("\nList of Documents with J(d1,d2) more than 0.5")
for x in range(0,50):
  file1 = "file" + getFileNo(x)
  for y in range(0,100):
    if estimateMatrix[x][y] > 0.5:
      file2 = "file" + getFileNo(y)
      if file1 != file2:
        shinglesSet1 = docShingleDict[file1]
        shinglesSet2 = docShingleDict[file2]
        print ("d1: " + file1 + " and d2: " + file2)
        print ("J(d1,d2): " + str(estimateMatrix[x][y])	)
        jaccard = (len(shinglesSet1.intersection(shinglesSet2)) / len(shinglesSet1.union(shinglesSet2)))
        print ("Jaccard coefficient: ", str(jaccard))
        #print 


    


List of Documents with J(d1,d2) more than 0.5
d1: file01 and d2: file75
J(d1,d2): 0.8
Jaccard coefficient:  0.8928571428571429
d1: file02 and d2: file76
J(d1,d2): 0.96
Jaccard coefficient:  0.9471153846153846
d1: file03 and d2: file87
J(d1,d2): 0.88
Jaccard coefficient:  0.9831932773109243
d1: file09 and d2: file64
J(d1,d2): 0.84
Jaccard coefficient:  0.7886740331491713
d1: file14 and d2: file88
J(d1,d2): 0.88
Jaccard coefficient:  0.9165354330708662
d1: file18 and d2: file99
J(d1,d2): 1.0
Jaccard coefficient:  0.9204152249134948
d1: file23 and d2: file51
J(d1,d2): 1.0
Jaccard coefficient:  0.9543057996485061
d1: file25 and d2: file40
J(d1,d2): 0.92
Jaccard coefficient:  0.96
d1: file32 and d2: file52
J(d1,d2): 0.96
Jaccard coefficient:  0.9531502423263328
d1: file34 and d2: file63
J(d1,d2): 0.96
Jaccard coefficient:  0.9702276707530648
d1: file40 and d2: file25
J(d1,d2): 0.92
Jaccard coefficient:  0.96


In [ ]:
docJaccard = {}

def pop(jaccardList):
  estimatedJaccardC, fileX = heappop(jaccardList)
  print (fileX)

print( "Three nearest neighbors for the first 10 files")

for x in range(0,10):
  file1 = "file" + getFileNo(x)
  estimatedJaccardList = []
  for y in range(0,100):
    file2 = "file" + getFileNo(y)
    if file1 != file2:
      heappush(estimatedJaccardList, (-estimateMatrix[x][y], file2))
  
  #print estimatedJaccardList
  print ("\n" + file1 + ":")
  for x in range(0,3):
    pop(estimatedJaccardList)

Three nearest neighbors for the first 10 files

file00:
file26
file19
file12

file01:
file75
file27
file20

file02:
file76
file28
file21

file03:
file87
file16
file29

file04:
file17
file23
file31

file05:
file18
file99
file78

file06:
file19
file26
file33

file07:
file20
file34
file63

file08:
file28
file21
file35

file09:
file64
file70
file22
